In [1]:
import numpy as np

In [ ]:
#!pip install plotly

In [2]:
import plotly.io as pio
dpi = 600
format = ".tif"
width = 2.0
height = 2.0
scale = 6
text_color = "black"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive

/content/drive


In [5]:
cd MyDrive

/content/drive/MyDrive


In [6]:
cd XBBvariants

/content/drive/MyDrive/XBBvariants


In [7]:
import os
cwd = os.getcwd() + os.sep
cwd

'/content/drive/MyDrive/XBBvariants/'

In [11]:
th_min = 4.0
th_max = 8.0

pdbFilesPath = cwd+"/Data/PDB/"

spike_muts_variants = {
                        "XBB.1": np.array([19, 24, 83, 142, 146, 183, 213, 252, 339, 346, 368, 371, 373, 375, 376, 405, 408, 417, 440, 445, 446,
                                              460, 477, 478, 484, 490, 498, 501, 505, 614, 655, 679, 681, 764, 796, 954, 969]),
                        "XBB.1.9.1": np.array([19, 24, 83, 142, 146, 183, 213, 252, 339, 346, 368, 371, 373, 375, 376, 405, 408, 417, 440, 445, 446,
                                              460, 477, 478, 484, 486, 490, 498, 501, 505, 614, 655, 679, 681, 764, 796, 954, 969]),
                       "XBB2.3": np.array([19, 24, 83, 142, 146, 183, 213, 253, 339, 346, 368, 371, 373, 375, 376, 405, 408, 417, 440, 445, 446,
                                              460, 477, 478, 484, 486, 490, 498, 501, 505, 521, 614, 655, 679, 681, 764, 796, 954, 969]),
                       "XBB.1.5": np.array([19, 24, 83, 142, 146, 183, 213, 252, 339, 346, 368, 371, 373, 375, 376, 405, 408, 417, 440, 445, 446,
                                              460, 477, 478, 484, 486, 490, 498, 501, 505, 614, 655, 679, 681, 764, 796, 954, 969]),
                       "XBB.1.16": np.array([19, 24, 83, 142, 146, 180, 183, 213, 252, 339, 346, 368, 371, 373, 375, 376, 405, 408, 417, 440, 445, 446,
                                              460, 477, 478, 484, 486, 490, 498, 501, 505, 614, 655, 679, 681, 764, 796, 954, 969]),
                       "EG.5.1": np.array([19, 24, 52, 83, 142, 146, 180, 183, 213, 252, 339, 346, 368, 371, 373, 375, 376, 405, 408, 417, 440, 445, 446,
                                            456, 460, 477, 478, 484, 486, 490, 498, 501, 505, 614, 655, 679, 681, 764, 796, 954, 969])
                      }


In [13]:
conformations = ["closed", "open", "complex"]

In [36]:
import pandas as pd

A_s = dict()

for conformation in conformations:
  A_s[conformation] = {}
  for i, pdb in enumerate(spike_muts_variants.keys()):
    #read PCN
    print(pdb)
    A = np.loadtxt(cwd+"/Data/Adj/{}_{}_adj_mat_{}_{}.txt".format(pdb.lower(), conformation, str(th_min), str(th_max)))
    A_s[conformation][pdb] = A

XBB.1
XBB.1.9.1
XBB2.3
XBB.1.5
XBB.1.16
EG.5.1
XBB.1
XBB.1.9.1
XBB2.3
XBB.1.5
XBB.1.16
EG.5.1
XBB.1
XBB.1.9.1
XBB2.3
XBB.1.5
XBB.1.16
EG.5.1


In [37]:
A_s.keys()

dict_keys(['closed', 'open', 'complex'])

In [18]:
def compute_contact_similarity(contact_sim_matrix):
    n, m = contact_sim_matrix.shape
    total = n*m
    n_diff_contacts = np.count_nonzero(contact_sim_matrix)
    return ((total-n_diff_contacts)/total)*100.0

In [33]:
def readPDBFile(pdbFilePath):

    atoms = []
    with open(pdbFilePath) as pdbfile:
        for line in pdbfile:
            if line[:4] == 'ATOM':
              # Split the line
                splitted_line = [line[:6], line[6:11], line[12:16], line[17:20], line[21], line[22:26], line[30:38], line[38:46], line[46:54], line[56:61], line[62:66]]
                atoms.append(splitted_line)
                # To format again the pdb file with the fields extracted
                #print("%-6s%5s %4s %3s %s %4s    %8s%8s%8s   %3s%3s\n"%tuple(splitted_line))
    return np.array(atoms)

def map_Graph(G, residue_names):

    mapping = dict()
    for i, residue_name in enumerate(residue_names):
        mapping[i] = residue_name

    #relabel nodes
    G = nx.relabel_nodes(G, mapping)
    return G

In [20]:
def plot_contact_sim(contact_sim, var1, var2):

    n = contact_sim.shape[0]
    fig = go.Figure(data=go.Heatmap(
                      z=contact_sim,
                      x=np.arange(0, n),
                      y=np.arange(0, n),
                      colorscale='OrRd'
                    )
                 )
    #fig.show()
    return fig

In [21]:
import plotly.graph_objects as go

In [40]:
def compute_contact_similarity_matrix(A1, A2):

    diff = A1 != A2
    contact_sim = np.zeros((diff.shape[0], diff.shape[1]))
    for i in range (diff.shape[0]):
        for j in range (diff.shape[1]):
            elem = diff[i][j]
            if elem: #contact difference
                contact_sim[i, j] = 1.0
            else:
                contact_sim[i, j] = 0.0
    return contact_sim

In [22]:
spikes = list(spike_muts_variants.keys())

In [59]:
import networkx as nx

n = len(spikes)
figures = {}

contacts_sim_all = {}
contacts_sim_pandas = {}
for conformation in conformations:
  contacts_sim_mat = {}
  contacts_sim_all[conformation] = {}
  contacts_sim_pandas[conformation] = np.zeros((n, n), dtype = float)
  for i, (var1, A1) in enumerate(A_s[conformation].items()):

      n1 = A1.shape[0]
      G_variant1 = nx.from_numpy_array(A1)
      var1 = var1.lower()
      for j, (var2, A2) in enumerate(A_s[conformation].items()):
          var2 = var2.lower()
          if i > j:
              if var1 != var2:
                  if "{}!={}".format(var1, var2) not in contacts_sim_mat.keys():
                      print(var1, var2)
                      n2 = A2.shape[0]

                      G_variant2 = nx.from_numpy_array(A2)

                      contact_sim_matrix = compute_contact_similarity_matrix(A1, A2)
                      contacts_sim_mat[var1+"!="+var2] = contact_sim_matrix
                      contact_sim = compute_contact_similarity(contact_sim_matrix)
                      contacts_sim_pandas[conformation][i, j] = contact_sim
                      contacts_sim_pandas[conformation][j, i] = contact_sim
                      print("Contact similarity between {} and {} Spike SARS-CoV-2 variants: {} %".format(var1, var2, contact_sim))
                      #fig = plot_contact_sim(contact_sim_matrix, var1, var2)
                      #figures[var1+"!="+var2] = fig
          elif i == j:
              print("i == j, Contact similarity between {} and {} Spike SARS-CoV-2 variants: {} %".format(var1, var2, 100.0))
              contacts_sim_pandas[conformation][i, j] = 100.0
              contacts_sim_pandas[conformation][j, i] = 100.0
  contacts_sim_all[conformation] = contacts_sim_mat

i == j, Contact similarity between xbb.1 and xbb.1 Spike SARS-CoV-2 variants: 100.0 %
xbb.1.9.1 xbb.1
Contact similarity between xbb.1.9.1 and xbb.1 Spike SARS-CoV-2 variants: 100.0 %
i == j, Contact similarity between xbb.1.9.1 and xbb.1.9.1 Spike SARS-CoV-2 variants: 100.0 %
xbb2.3 xbb.1
Contact similarity between xbb2.3 and xbb.1 Spike SARS-CoV-2 variants: 100.0 %
xbb2.3 xbb.1.9.1
Contact similarity between xbb2.3 and xbb.1.9.1 Spike SARS-CoV-2 variants: 100.0 %
i == j, Contact similarity between xbb2.3 and xbb2.3 Spike SARS-CoV-2 variants: 100.0 %
xbb.1.5 xbb.1
Contact similarity between xbb.1.5 and xbb.1 Spike SARS-CoV-2 variants: 100.0 %
xbb.1.5 xbb.1.9.1
Contact similarity between xbb.1.5 and xbb.1.9.1 Spike SARS-CoV-2 variants: 100.0 %
xbb.1.5 xbb2.3
Contact similarity between xbb.1.5 and xbb2.3 Spike SARS-CoV-2 variants: 100.0 %
i == j, Contact similarity between xbb.1.5 and xbb.1.5 Spike SARS-CoV-2 variants: 100.0 %
xbb.1.16 xbb.1
Contact similarity between xbb.1.16 and xbb.1

In [ ]:
A_s

In [47]:
import pandas as pd
for conformation in conformations:
  df_cs = pd.DataFrame(contacts_sim_pandas[conformation], columns = list(A_s[conformation].keys()), index = list(A_s[conformation].keys()))
  df_cs.to_csv(cwd+"/Data/contact_sim_{}_variants.csv".format(conformation))

In [48]:
df_cs

,XBB.1,XBB.1.9.1,XBB2.3,XBB.1.5,XBB.1.16,EG.5.1
XBB.1,100.0,100.0,100.0,100.0,100.0,100.0
XBB.1.9.1,100.0,100.0,100.0,100.0,100.0,100.0
XBB2.3,100.0,100.0,100.0,100.0,100.0,100.0
XBB.1.5,100.0,100.0,100.0,100.0,100.0,100.0
XBB.1.16,100.0,100.0,100.0,100.0,100.0,100.0
EG.5.1,100.0,100.0,100.0,100.0,100.0,100.0


In [ ]:
#!pip install -U kaleido

In [ ]:
#!pip install --upgrade nbformat

In [51]:
#plot contact sim
import plotly.figure_factory as ff
from scipy.spatial.distance import pdist, squareform

conformation = "closed"
contacts_sim = contacts_sim_pandas[conformation]
ticksuffix = "                             "
# Initialize figure by creating upper dendrogram
labels = np.array(sorted(list(spike_muts_variants.keys())))
labels_no_mutant = np.array(sorted([elem for elem in list(spike_muts_variants.keys()) if "Mutant" not in elem]))
i_no_mutant = [i for i, elem in enumerate(list(spike_muts_variants.keys())) if "Mutant" not in elem]
n_no_mut = labels_no_mutant.shape[0]
n = labels.shape[0]
contacts_sim_nomutant = np.zeros((n_no_mut, n_no_mut))
idx=0
for i in range(n):
    jdx=0
    if i in i_no_mutant:
        for j in range(n):
            if j in i_no_mutant:
                contacts_sim_nomutant[idx, jdx] = contacts_sim[i, j]
                jdx+=1
        idx+=1
print(contacts_sim_nomutant)

fig = ff.create_dendrogram(contacts_sim_nomutant, orientation='bottom', labels=labels_no_mutant)
for i in range(len(fig['data'])):
    fig['data'][i]['yaxis'] = 'y2'

# Create Side Dendrogram
dendro_side = ff.create_dendrogram(contacts_sim_nomutant, orientation='right')
for i in range(len(dendro_side['data'])):
    dendro_side['data'][i]['xaxis'] = 'x2'

# Add Side Dendrogram Data to Figure
for data in dendro_side['data']:
    fig.add_trace(data)

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = [1-contacts_sim_nomutant[i, j] for i in range(contacts_sim_nomutant.shape[0]) for j in range(contacts_sim_nomutant.shape[1]) if i > j]  #pdist(contacts_sim_nomutant)
heat_data = contacts_sim_nomutant #squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = [1-contacts_sim_nomutant[i, j] for i in range(contacts_sim_nomutant.shape[0]) for j in range(contacts_sim_nomutant.shape[1]) if i > j]  #pdist(contacts_sim_nomutant)
heat_data = contacts_sim_nomutant #squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

heatmap = [
    go.Heatmap(
        x = dendro_leaves,
        y = dendro_leaves,
        z = heat_data,
        colorscale = 'Blues'
    )
]

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

# Add Heatmap Data to Figure
for data in heatmap:
    fig.add_trace(data)

# Edit Layout
fig.update_layout({'width':width*dpi, 'height':height*dpi,
                         'showlegend':False, 'hovermode': 'closest', 'title_font_color': text_color, 'font_color': text_color
                         })
# Edit xaxis
fig.update_layout(xaxis={'domain': [.15, 1],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'tickmode': "array",
                                  'tickvals': np.arange(5, 215, 10),
                                  'ticktext': "<b>"+fig['layout']['xaxis']['ticktext']+"</b>"+ticksuffix})
# Edit xaxis2
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': True,
                                  'tickmode': "array",
                                  'tickvals': np.arange(5, 215, 10),
                                  'ticktext': "<b>"+fig['layout']['xaxis']['ticktext']+"</b>"+ticksuffix
                        })
# Edit yaxis2
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks': ""})

# Plot!
fig.show()

#Save
#fig.write_html("Figures/ContactSimilarity/contactDistNormNoMut.html")
#pio.write_image(fig, "Figures/ContactSimilarity/contactDistNormNoMut.png", engine = "orca", width=width*dpi, height=height*dpi, scale=scale)

[[100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100. 100.]]


In [52]:
#plot contact sim
import plotly.figure_factory as ff
from scipy.spatial.distance import pdist, squareform

ticksuffix = "                             "
# Initialize figure by creating upper dendrogram
labels = np.array(sorted(list(spike_muts_variants.keys())))
n = labels.shape[0]

fig = ff.create_dendrogram(contacts_sim, orientation='bottom', labels=labels)
for i in range(len(fig['data'])):
    fig['data'][i]['yaxis'] = 'y2'

# Create Side Dendrogram
dendro_side = ff.create_dendrogram(contacts_sim, orientation='right')
for i in range(len(dendro_side['data'])):
    dendro_side['data'][i]['xaxis'] = 'x2'

# Add Side Dendrogram Data to Figure
for data in dendro_side['data']:
    fig.add_trace(data)

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = [1-contacts_sim[i, j] for i in range(contacts_sim.shape[0]) for j in range(contacts_sim.shape[1]) if i > j]  #pdist(contacts_sim)
heat_data = contacts_sim #squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = [1-contacts_sim[i, j] for i in range(contacts_sim.shape[0]) for j in range(contacts_sim.shape[1]) if i > j]  #pdist(contacts_sim)
heat_data = contacts_sim #squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

heatmap = [
    go.Heatmap(
        x = dendro_leaves,
        y = dendro_leaves,
        z = heat_data,
        colorscale = 'Blues'
    )
]

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

# Add Heatmap Data to Figure
for data in heatmap:
    fig.add_trace(data)

# Edit Layout
fig.update_layout({'width':width*dpi, 'height':height*dpi,
                         'showlegend':False, 'hovermode': 'closest', 'title_font_color': text_color, 'font_color': text_color
                         })
# Edit xaxis
fig.update_layout(xaxis={'domain': [.15, 1],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'tickmode': "array",
                                  'tickvals': np.arange(5, 215, 10),
                                  'ticktext': "<b>"+fig['layout']['xaxis']['ticktext']+"</b>"+ticksuffix})
# Edit xaxis2
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': True,
                                  'tickmode': "array",
                                  'tickvals': np.arange(5, 215, 10),
                                  'ticktext': "<b>"+fig['layout']['xaxis']['ticktext']+"</b>"+ticksuffix,
                        })
# Edit yaxis2
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks': ""})

# Plot!
fig.show()

#Save
#fig.write_html("Figures/ContactSimilarity/contactDistNorm.html")
#pio.write_image(fig, "Figures/ContactSimilarity/contactDistNorm.png", engine = "orca", width=width*dpi, height=height*dpi, scale=scale)

In [53]:
N = contacts_sim.shape[0]
labels = list(spike_muts_variants.keys())
labels = ["<b>{}</b>".format(label) for label in labels]
fig = go.Figure(data=go.Heatmap(
                      z=contacts_sim,
                      x=np.arange(0, N),
                      y=np.arange(0, N),
                      colorscale='OrRd'
                    )
                 )
# Edit yaxis
fig.update_layout(yaxis={         'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': True,
                                  'ticklabelposition': "outside bottom",
                                  'tickmode': "array",
                                  'tickvals': np.arange(0, N, 1),
                                  'ticktext': labels,
                                  'tickfont_size': 20
                        })
# Edit xaxis
fig.update_layout(xaxis={

                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': True,
                                  'ticklabelposition': "outside bottom",
                                  'tickmode': "array",
                                  'tickvals': np.arange(0, N, 1),
                                  'ticktext': labels,
                                  'tickfont_size': 20


                        })
fig.update_layout(width=width*dpi, height=height*dpi, title_font_color = text_color, font_color = text_color)
fig.show()

#Save
#fig.write_html("Figures/ContactSimilarity/contactSim.html")
#pio.write_image(fig, "Figures/ContactSimilarity/contactSim.png", engine = "orca", width=width*dpi, height=height*dpi, scale=scale)

In [60]:
var1 = "xbb.1"
var2 = "eg.5.1"
contacts_sim_mat = contacts_sim_all["closed"]

if var1+"!="+var2 in contacts_sim_mat.keys():
    key = var1+"!="+var2
elif var2+"!="+var1 in contacts_sim_mat.keys():
    key = var2+"!="+var1
print(key)
fig = plot_contact_sim(contacts_sim_mat[key], var1, var2)
fig.show()

eg.5.1!=xbb.1


In [61]:
contacts_sim_mat.keys()

In [ ]:
curr_min = 100.0
curr_i_min = 0
curr_j_min = 0
for i in range(n):
    for j in range(n):
        curr_value = contacts_sim_mat[i][j]#contacts_sim[i][j]
        if curr_value > 0.0:
            if curr_value < curr_min:
                curr_min = curr_value
                curr_i_min = i
                curr_j_min = j

print("Min {} found for i = {} and j = {}".format(curr_min, curr_i_min, curr_j_min))
print(curr_i_min, spikes[curr_i_min])
print(curr_j_min, spikes[curr_j_min])